# MCP: Model Context Protocol — Zero to Hero

## Part 1: The Problem — Why Did MCP Even Exist?

Let me start with a story.

---

### The World Before MCP

Imagine you hire a brilliant new employee — let's call them Alex. Alex is incredibly smart, can write code, analyze data, draft emails, answer questions. But Alex has one bizarre limitation: **they can only work with information you physically hand them on paper.**

Alex cannot:
- Open your company's database themselves
- Check your calendar
- Read files from your computer
- Search the internet
- Query your CRM

Every single time Alex needs external information, *you* have to go get it, print it out, and hand it to them. Then Alex does their work. Then if they need something else, you go get that too.

**That's exactly what LLMs (like GPT-4, Claude, etc.) were like before MCP.**

They were brilliant minds trapped in a box. They could only work with text you pasted into the chat window.

---

### The "1000 Integrations" Problem

Now let's say you want to build a smarter AI assistant for your company. You want it to:
- Read from your PostgreSQL database
- Write to your Notion workspace
- Query GitHub issues
- Check Slack messages
- Look at your Google Calendar

Before MCP, every single AI company, every single developer, every single product had to **build custom integrations for each of these tools, for each of their AI models.**

So Anthropic builds Claude → they write a GitHub connector for Claude.  
OpenAI builds GPT → they write a GitHub connector for GPT.  
Some startup builds their AI product → they write *another* GitHub connector.

GitHub itself gets called by 500 different teams in 500 different incompatible ways.

This is the **M × N problem**: M models × N tools = M×N custom integrations. It scales terribly.

```
Claude ──── custom code ───→ GitHub
Claude ──── custom code ───→ Notion  
Claude ──── custom code ───→ Postgres
GPT-4  ──── custom code ───→ GitHub  ← same thing, built again!
GPT-4  ──── custom code ───→ Notion  ← same thing, built again!
Gemini ──── custom code ───→ GitHub  ← same thing, built again!
```

Every integration is a snowflake. Nothing is reusable. It's a maintenance nightmare.

---

### The Real-World Pain

Here's a concrete scenario. You're a developer and you want Claude to help you debug your app. You want Claude to:

1. Look at the error logs in your server
2. Check the relevant code in your GitHub repo
3. Look at recent deployments in your CI/CD system
4. Suggest a fix

**Without MCP:** You manually copy-paste logs, then copy-paste code, then describe the deployment. You're the middleman. Claude is a brain with no hands.

**With MCP:** Claude reaches out to your log server, your GitHub, your CI/CD system — all by itself — and comes back with a full diagnosis. You just asked the question.

---

## Part 2: What is MCP?

**MCP = Model Context Protocol**

It's an **open standard protocol** (like HTTP, but for AI tools) created by Anthropic in late 2024. It defines a *universal language* for how AI models talk to external tools, data sources, and services.

The key insight: **instead of M×N custom integrations, you build M model connectors + N tool servers = M+N total work.**

```
Claude ──── MCP ────→ [ MCP Server: GitHub ]
GPT-4  ──── MCP ────→ [ MCP Server: GitHub ]  ← same server, reused!
Gemini ──── MCP ────→ [ MCP Server: GitHub ]  ← same server, reused!
                       [ MCP Server: Notion ]
                       [ MCP Server: Postgres ]
```

GitHub only needs to build *one* MCP server. Every AI model that speaks MCP can use it automatically. This is the USB-C moment for AI integrations.

---

## Part 3: The Architecture — How It Actually Works

MCP follows a **client-server architecture**. There are three main players:

```
┌─────────────────────────────────────────┐
│              HOST APPLICATION           │
│  (Claude Desktop, Cursor, your app)     │
│                                         │
│   ┌─────────────┐                       │
│   │ MCP CLIENT  │◄──── talks to ──────► │ MCP SERVER
│   └─────────────┘                       │ (GitHub, Postgres, etc.)
│         ▲                               │
│         │                               │
│   ┌─────▼──────┐                        │
│   │    LLM     │                        │
│   └────────────┘                        │
└─────────────────────────────────────────┘
```

Let's break each piece down:

---

### 🏠 The Host

The **host** is the application the user is actually running. Examples:
- Claude Desktop (Anthropic's desktop app)
- Cursor (AI code editor)
- Your own custom AI application

The host is responsible for:
- Running the LLM
- Managing MCP clients
- Presenting results to the user
- Enforcing security and permissions

---

### 📱 The MCP Client

The **client** lives *inside* the host application. It's the bridge between the LLM and the outside world.

Think of the client as Alex's personal assistant. When Alex (the LLM) says "I need to check the GitHub repo", the client is the one who actually goes and does it.

Each client maintains a **1-to-1 connection** with one MCP server.

Responsibilities:
- Connect to MCP servers
- Discover what capabilities each server offers
- Send requests to servers on behalf of the LLM
- Return results back to the LLM

---

### 🖥️ The MCP Server

The **server** is the thing that actually has access to external resources. It's a lightweight process that wraps a tool or data source and exposes it in MCP's standard format.

Examples of MCP servers:
- A GitHub MCP server (can read/write repos, issues, PRs)
- A PostgreSQL MCP server (can run queries)
- A filesystem MCP server (can read/write files on your computer)
- A Slack MCP server (can read/post messages)
- A web search MCP server (can search the internet)

The server exposes three types of things to clients:

---

## Part 4: The Three Primitives — What Servers Actually Expose

This is the core of MCP. Every server can expose up to three types of capabilities:

### 1. 🔧 Tools
**What:** Functions the LLM can *call* to take actions or get computed results.  
**Analogy:** Tools are like API endpoints — "do this thing."  
**Examples:**
- `search_github_issues(query, repo)` → returns list of issues
- `run_sql_query(sql)` → returns query results
- `send_slack_message(channel, text)` → sends a message
- `create_file(path, content)` → creates a file

Tools are **model-controlled** — the LLM decides when to call them and with what arguments. This is the most powerful primitive.

---

### 2. 📄 Resources
**What:** Data that the server exposes for the LLM to *read*.  
**Analogy:** Resources are like files or URLs — "here's some data."  
**Examples:**
- The contents of a file: `file:///home/user/project/main.py`
- A database schema: `postgres://mydb/schema`
- A webpage: `https://docs.example.com/api`

Resources are **application-controlled** — the host decides which resources to include in the LLM's context. The LLM doesn't directly call resources; the app decides what to include.

---

### 3. 💬 Prompts
**What:** Pre-written prompt templates that the server provides.  
**Analogy:** Prompts are like saved macros or slash commands — "use this template."  
**Examples:**
- A "debug this code" prompt that automatically structures a debugging workflow
- A "summarize this PR" prompt
- A "write a commit message" prompt

Prompts are **user-controlled** — the user explicitly picks which prompt template to invoke.

---

## Part 5: The Communication Protocol — How Do They Talk?

MCP uses **JSON-RPC 2.0** as its message format. This is a simple, well-established standard where:
- Messages are JSON objects
- Each request has a method name and parameters
- Responses return results or errors

### Transport Layers

MCP supports two transport mechanisms:

**1. Stdio (Standard Input/Output)**  
Used when the server runs as a local process on the same machine. The client launches the server as a subprocess and communicates via stdin/stdout pipes.

```
Host App → spawns → MCP Server process
         ←─────── JSON messages via stdin/stdout ──────→
```

This is the most common setup for local tools (filesystem access, local databases, local dev tools).

**2. HTTP + SSE (Server-Sent Events)**  
Used when the server is remote — running on a different machine or in the cloud.

```
Host App ──── HTTP POST ───→ Remote MCP Server
         ←─── SSE stream ───
```

SSE allows the server to stream responses back, which is useful for long-running operations.

---

### The Lifecycle of a Request

Here's exactly what happens when you ask an AI assistant something that requires a tool:

```
You: "What are the open issues in my GitHub repo?"

1. HOST receives your message, passes it to the LLM

2. LLM thinks: "I need to check GitHub. Let me call the 
   search_issues tool."
   
3. LLM outputs a tool call:
   {
     "tool": "search_github_issues",
     "arguments": { "repo": "myuser/myrepo", "state": "open" }
   }

4. MCP CLIENT receives this tool call

5. CLIENT sends to MCP SERVER (GitHub server):
   {
     "jsonrpc": "2.0",
     "method": "tools/call",
     "params": {
       "name": "search_github_issues",
       "arguments": { "repo": "myuser/myrepo", "state": "open" }
     }
   }

6. MCP SERVER calls the actual GitHub API with your credentials

7. GitHub API returns data to the MCP server

8. MCP SERVER sends the result back to the CLIENT:
   {
     "jsonrpc": "2.0",
     "result": {
       "content": [{ "type": "text", "text": "Issue #42: Bug in login..." }]
     }
   }

9. CLIENT passes result back to the LLM

10. LLM formulates its response using this real data

11. You see: "You have 3 open issues: #42 Bug in login, #43 
    Performance issue, #44 Docs missing..."
```

---

## Part 6: The Connection Handshake — Initialization

Before any of the above can happen, the client and server need to introduce themselves. Here's how:

**Step 1: Initialize**
```json
Client → Server:
{
  "method": "initialize",
  "params": {
    "protocolVersion": "2024-11-05",
    "clientInfo": { "name": "Claude Desktop", "version": "1.0" },
    "capabilities": { "sampling": {} }
  }
}
```

**Step 2: Server responds with its capabilities**
```json
Server → Client:
{
  "result": {
    "protocolVersion": "2024-11-05",
    "serverInfo": { "name": "github-mcp-server", "version": "0.1" },
    "capabilities": {
      "tools": { "listChanged": true },
      "resources": { "subscribe": true }
    }
  }
}
```

**Step 3: Initialized notification**
```json
Client → Server:
{ "method": "notifications/initialized" }
```

Now they're connected. The client can start asking "what tools do you have?" and the server will list everything it offers.

---

## Part 7: Discovery — How Does the LLM Know What Tools Exist?

When a client connects to a server, it asks for a list of everything available:

**List Tools:**
```json
Client → Server: { "method": "tools/list" }

Server → Client:
{
  "tools": [
    {
      "name": "search_issues",
      "description": "Search GitHub issues in a repository",
      "inputSchema": {
        "type": "object",
        "properties": {
          "repo": { "type": "string", "description": "owner/repo format" },
          "query": { "type": "string", "description": "search query" }
        },
        "required": ["repo"]
      }
    },
    {
      "name": "create_issue",
      "description": "Create a new GitHub issue",
      ...
    }
  ]
}
```

This tool list gets injected into the LLM's context (system prompt). The LLM now *knows* these tools exist and understands how to call them. This is how the LLM learns what it can do at runtime — it's dynamic, not hardcoded.

---

## Part 8: Security — This Seems Dangerous?

You're right to wonder. Giving an AI model the ability to call tools that can read your files, write to databases, and send messages is powerful and potentially scary.

MCP has several security mechanisms:

**1. User Consent**  
Hosts must present tool calls to users and get approval before executing, especially for destructive operations. Claude Desktop, for example, shows you "Claude wants to use the filesystem tool — Allow?"

**2. Sandboxing**  
Each MCP server only has access to what it's configured for. A filesystem server might only have access to your project folder, not your entire computer.

**3. Local by Default**  
Most MCP servers run locally on your machine (via stdio), so no data goes to a third-party server. Your credentials never leave your machine.

**4. Scoped Credentials**  
Servers use scoped API tokens with minimal necessary permissions. A GitHub server might only have read access.

**5. The LLM Doesn't Execute — It Requests**  
The LLM cannot directly execute anything. It can only *request* a tool call. The host/client decides whether to honor that request. This is an important architectural decision — humans remain in control.

---

## Part 9: Building Your Own MCP Server — Practical

Let's make this concrete. Here's how you'd build a simple MCP server in Python using the official SDK:

```python
from mcp.server import Server
from mcp.server.stdio import stdio_server
from mcp import types

# Create the server
app = Server("my-todo-server")

# In-memory todo list
todos = []

# Define a tool: add a todo
@app.list_tools()
async def list_tools():
    return [
        types.Tool(
            name="add_todo",
            description="Add a new todo item",
            inputSchema={
                "type": "object",
                "properties": {
                    "task": {"type": "string", "description": "The task to add"}
                },
                "required": ["task"]
            }
        ),
        types.Tool(
            name="list_todos",
            description="List all todo items",
            inputSchema={"type": "object", "properties": {}}
        )
    ]

# Handle tool calls
@app.call_tool()
async def call_tool(name: str, arguments: dict):
    if name == "add_todo":
        task = arguments["task"]
        todos.append(task)
        return [types.TextContent(type="text", text=f"Added: {task}")]
    
    elif name == "list_todos":
        if not todos:
            return [types.TextContent(type="text", text="No todos yet!")]
        todo_list = "\n".join(f"- {t}" for t in todos)
        return [types.TextContent(type="text", text=todo_list)]

# Run the server
async def main():
    async with stdio_server() as (read, write):
        await app.run(read, write, app.create_initialization_options())

if __name__ == "__main__":
    import asyncio
    asyncio.run(main())
```

That's it. You just built an MCP server. Now any MCP-compatible AI client can use it.

---

## Part 10: Configuring MCP in a Host (Claude Desktop Example)

Once you have an MCP server, you tell your host app about it. For Claude Desktop, you edit a config file:

```json
{
  "mcpServers": {
    "my-todo-server": {
      "command": "python",
      "args": ["/path/to/my_todo_server.py"]
    },
    "github": {
      "command": "npx",
      "args": ["-y", "@modelcontextprotocol/server-github"],
      "env": {
        "GITHUB_TOKEN": "ghp_your_token_here"
      }
    },
    "filesystem": {
      "command": "npx",
      "args": [
        "-y",
        "@modelcontextprotocol/server-filesystem",
        "/Users/you/projects"
      ]
    }
  }
}
```

Claude Desktop reads this, launches each server as a subprocess, connects via stdio, and Claude now has access to all these tools.

---

## Part 11: Sampling — The Reverse Direction

Here's an advanced concept most people miss: **MCP isn't just one-directional.**

In normal flow: Host/LLM → Client → Server (LLM drives everything).

But servers can also ask the *LLM* to do things. This is called **Sampling**.

**Use case:** Imagine a complex MCP server that needs to summarize 100 documents. Rather than the server implementing its own summarization logic, it can ask the host's LLM to help.

```
MCP Server → Client → Host → LLM: "Please summarize this text"
                            LLM → Client → Server: "Here's the summary"
```

This enables **agentic loops** where servers themselves become orchestrators, spinning up sub-tasks for the LLM. It's how you build complex multi-step AI agents with MCP.

---

## Part 12: MCP vs Other Things — Clarifying Confusion

**MCP vs Function Calling (OpenAI's tool use):**  
Function calling is the LLM's ability to request a tool call. MCP is the *transport and discovery layer* for those tools. They work together — MCP tools get exposed to the LLM *as* function calls.

**MCP vs LangChain tools:**  
LangChain tools are Python functions registered in LangChain's framework. They're not portable — they only work in LangChain apps. MCP tools are language-agnostic, framework-agnostic, and any compatible host can use them.

**MCP vs REST APIs:**  
REST APIs require custom integration code for every caller. MCP is a *standard* so any MCP client can talk to any MCP server without custom code, just like how any browser can load any HTTP website.

**MCP vs Plugins (ChatGPT plugins):**  
ChatGPT plugins were OpenAI-specific, required their approval, and were shut down. MCP is open, decentralized — anyone can build a server, anyone can use it.

---

## Part 13: The Ecosystem Today

The MCP ecosystem has exploded. There are hundreds of community-built MCP servers for:

**Developer Tools:** GitHub, GitLab, Jira, Linear, Sentry  
**Databases:** PostgreSQL, MySQL, SQLite, MongoDB, Redis  
**Productivity:** Notion, Google Docs, Obsidian, Todoist  
**Communication:** Slack, Discord, Gmail, Outlook  
**Cloud:** AWS, GCP, Azure, Terraform  
**Local:** Filesystem, Terminal/Shell, Browser automation  
**Data:** Pandas, Excel, CSV processing  

Hosts that support MCP: Claude Desktop, Cursor, Zed editor, Continue.dev, Cline, Windsurf, and hundreds of custom apps.

---

## Part 14: The Mental Model — Summary

Here's the complete picture in your head:

```
┌─────────────────────────────────────────────────────────────┐
│  THE WORLD BEFORE MCP: Every integration is custom          │
│                                                             │
│  LLM-A ──custom──► Tool-1                                   │
│  LLM-A ──custom──► Tool-2   ← snowflakes, unmaintainable   │
│  LLM-B ──custom──► Tool-1                                   │
└─────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────┐
│  THE WORLD WITH MCP: Standard protocol like USB-C           │
│                                                             │
│  ┌──────────┐      ┌──────────────┐      ┌─────────────┐   │
│  │  User    │      │     HOST     │      │ MCP Server  │   │
│  │          │◄────►│  (App + LLM) │◄────►│  (GitHub)   │   │
│  └──────────┘      │  MCP Client  │      └─────────────┘   │
│                    └──────┬───────┘      ┌─────────────┐   │
│                           └─────────────►│ MCP Server  │   │
│                                          │ (Postgres)  │   │
│                                          └─────────────┘   │
│                                          ┌─────────────┐   │
│                                          │ MCP Server  │   │
│                                          │ (Slack)     │   │
│                                          └─────────────┘   │
└─────────────────────────────────────────────────────────────┘
```

The LLM is now a **brain with hands.** It can reach out into the world, read data, take actions, and come back with real results — all through a standardized, secure, composable protocol.

---

## TL;DR — The Core Ideas

| Concept | Plain English |
|---|---|
| **MCP** | USB-C standard for connecting AI brains to tools |
| **Host** | The app the user runs (Claude Desktop, Cursor) |
| **Client** | The bridge inside the host that talks to servers |
| **Server** | A wrapper around a tool/data source that speaks MCP |
| **Tools** | Actions the LLM can invoke (functions) |
| **Resources** | Data the LLM can read (files, URLs, database rows) |
| **Prompts** | Reusable prompt templates the server provides |
| **Transport** | Stdio for local, HTTP+SSE for remote |
| **Protocol** | JSON-RPC 2.0 messages |
| **Security** | Local-first, user consent, scoped permissions |

MCP is essentially the **TCP/IP moment for AI agents** — a foundational protocol that makes everything interoperable, so the ecosystem can grow without everyone rebuilding the same integrations over and over. It's what turns an AI from a chatbot into an *agent*.